# Question 7 - OQ – How does the code handle corrupted image files?

To ensure robustness, the analysis pipeline should handle corrupted or unreadable image files.

For this, we can wrap the image loading logic in a `try/except` block inside the `analyze_batch()` function:

```python
try:
    img = np.load(filepath)
except Exception as e:
    print(f"⚠️ Skipping corrupted file {filename}: {e}")
    continue
